# CyberIntent-AI: End-to-End Demo Scenario

This notebook demonstrates the complete CyberIntent-AI workflow from data ingestion to threat response.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("CyberIntent-AI Complete Demo")
print("="*60)
print(f"Started at: {datetime.now()}")

## Step 1: Data Ingestion

In [ ]:
# Load network event data
df = pd.read_csv('data/sample_logs.csv')

print("\n[1] DATA INGESTION")
print(f"\n  ✓ Loaded {len(df)} network events")
print(f"  ✓ Features: {list(df.columns)}")
print(f"  ✓ Data types: {dict(df.dtypes)}")

## Step 2: Feature Engineering

In [ ]:
# Create engineered features
df['total_bytes'] = df['bytes_sent'] + df['bytes_received']
df['bytes_ratio'] = df['bytes_sent'] / (df['bytes_received'] + 1)
df['login_fail_rate'] = df['failed_logins'] / (df['successful_logins'] + 1)
df['packet_rate'] = df['bytes_sent'] / (df['duration'] + 0.01)

print("\n[2] FEATURE ENGINEERING")
engineered_features = ['total_bytes', 'bytes_ratio', 'login_fail_rate', 'packet_rate']
print(f"\n  ✓ Created {len(engineered_features)} new features")
for feat in engineered_features:
    print(f"    - {feat}")
print(f"\n  Total features available: {len(df.columns)}")

## Step 3: Anomaly Detection

# Analyze anomalies
anomalies = df[df['anomaly_score'] > 0.7]
normal = df[df['anomaly_score'] <= 0.7]

print("\n[3] ANOMALY DETECTION")
print(f"\n  ✓ Anomalies detected: {len(anomalies)} / {len(df)} ({len(anomalies)/len(df)*100:.1f}%)")
print(f"  ✓ Normal events: {len(normal)} / {len(df)} ({len(normal)/len(df)*100:.1f}%)")
print(f"\n  Anomaly statistics:")
print(f"    - Min score: {df['anomaly_score'].min():.3f}")
print(f"    - Max score: {df['anomaly_score'].max():.3f}")
print(f"    - Mean score: {df['anomaly_score'].mean():.3f}")
print(f"    - Median score: {df['anomaly_score'].median():.3f}")

## Step 4: Intent Prediction

print("\n[4] ATTACK INTENT PREDICTION")
print(f"\n  Intent distribution:")
for intent, count in df['intent_label'].value_counts().items():
    print(f"    - {intent}: {count} ({count/len(df)*100:.1f}%)")

# Threat analysis
threats = df[df['intent_label'] != 'benign']
print(f"\n  ✓ Threats identified: {len(threats)}")
print(f"  ✓ Threat types: {threats['intent_label'].nunique()}")

## Step 5: Risk Assessment

In [ ]:
# Analyze risk scores and create risk levels
df['risk_score'] = df['anomaly_score'] * 100  # Simplified risk calculation

def get_risk_level(score):
    if score < 25:
        return 'LOW'
    elif score < 50:
        return 'MEDIUM'
    elif score < 75:
        return 'HIGH'
    else:
        return 'CRITICAL'

df['risk_level'] = df['risk_score'].apply(get_risk_level)

print("\n[5] RISK ASSESSMENT")
print(f"\n  Risk level distribution:")
for level in ['LOW', 'MEDIUM', 'HIGH', 'CRITICAL']:
    count = (df['risk_level'] == level).sum()
    print(f"    - {level}: {count} ({count/len(df)*100:.1f}%)")

critical_events = df[df['risk_level'] == 'CRITICAL']
print(f"\n  ⚠️  Critical events requiring response: {len(critical_events)}")

# Generate alerts
alerts = []
for idx, row in df[df['anomaly_score'] > 0.7].iterrows():
    alert = {
        'id': f"ALERT-{idx:04d}",
        'type': row['intent_label'].upper(),
        'severity': row['risk_level'],
        'source_ip': row['src_ip'],
        'destination_ip': row['dst_ip'],
        'timestamp': row['timestamp'],
        'anomaly_score': row['anomaly_score']
    }
    alerts.append(alert)

alerts_df = pd.DataFrame(alerts)

print("\n[6] ALERT GENERATION")
print(f"\n  ✓ Generated {len(alerts_df)} alerts")
print(f"\n  Alert breakdown by severity:")
for severity in ['CRITICAL', 'HIGH', 'MEDIUM', 'LOW']:
    count = (alerts_df['severity'] == severity).sum()
    if count > 0:
        print(f"    - {severity}: {count}")

print(f"\n  Top 3 alerts:")
for idx, (_, row) in enumerate(alerts_df.nlargest(3, 'anomaly_score').iterrows()):
    print(f"    {idx+1}. {row['id']} - {row['type']} ({row['severity']})")
    print(f"       Source: {row['source_ip']} → {row['destination_ip']}")
    print(f"       Score: {row['anomaly_score']:.3f}")

## Step 7: Recommended Response Actions